In [1]:
import itertools as it
import time
from utils import *
from pysat.solvers import Glucose3

import numpy as np
from scipy import sparse

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
# from torch_geometric.nn import GATConv
from torch_geometric.data import Data

In [2]:
class GCN(torch.nn.Module):
    def __init__(self, node_features):
        super().__init__()
        # GCN initialization
        self.conv1 = GCNConv(node_features, 128)
        self.conv2 = GCNConv(128, 128)
        # self.conv1 = GATConv(node_features, 64, 5)
        # self.conv2 = GATConv(64 * 5, 128)
        # self.conv3 = GCNConv(128, 128)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = F.elu(x)
        # x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        # x = F.tanh(x)
        # x = self.conv3(x, edge_index)

        return x


In [3]:
sat_name = 'mrpp_4x4#4_5.processed.cnf'
# sat_name = 'countbitsrotate016.processed.cnf'

sat_path = f'./dataset/formulas/{sat_name}'
num_vars, num_clauses, sat_instance = read_sat(sat_path)
max_len = max([len(clause) for clause in sat_instance])

lig_adjacency_matrix, lig_weighted_adjacency_matrix = sat_to_lig_adjacency_matrix(sat_instance, num_vars)





# graph = nx.from_numpy_matrix(lig_adjacency_matrix)
# edges = nx.to_edgelist(graph)
# print(lig_adjacency_matrix.nonzero())

edge_index = torch.tensor(np.array(lig_adjacency_matrix.nonzero()), dtype=torch.long)
print(edge_index.shape)
edge_value = lig_weighted_adjacency_matrix[lig_adjacency_matrix.nonzero()]

embeddings = torch.load(f'./model/embeddings/{sat_name}.pt')
embeddings.requires_grad = False
# print(embeddings)
x = embeddings
data = Data(x=x, edge_index=edge_index)


torch.Size([2, 14472])


In [4]:
# training
model = GCN(50)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(500):
    optimizer.zero_grad()
    out = model(data)
    src, dst = edge_index
    score = (out[src] * out[dst]).sum(dim=-1)
    # score = torch.sigmoid(score)
    loss = F.mse_loss(score, torch.tensor(edge_value, dtype=torch.float))
    loss.backward()
    optimizer.step()
    # print(f'epoch: {epoch}, loss: {loss.item()}')

out = model(data)
src, dst = edge_index
score = (out[src] * out[dst]).sum(dim=-1)
# print(min(score))
# score = torch.sigmoid(score)
print(f"edge_value: {edge_value}")
print(f"score: {score.detach()}")
print(f"min score: {min(score)}")
print(f"max score: {max(score)}")

edge_value: [1. 1. 1. ... 2. 3. 2.]
score: tensor([1.5412, 0.8578, 0.8736,  ..., 2.2472, 1.7543, 1.1899])
min score: -1.7567486763000488
max score: 22.947275161743164


In [5]:
# CELL
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz

import torch

from cell.utils import link_prediction_performance
from cell.cell import Cell, EdgeOverlapCriterion, LinkPredictionCriterion
from cell.graph_statistics import compute_graph_statistics


sparse_matrix = sparse.csr_matrix(lig_adjacency_matrix)
cell_model = Cell(A=sparse_matrix,
             H=12,
             callbacks=[EdgeOverlapCriterion(invoke_every=10, edge_overlap_limit=.80)])


cell_model.train(steps=400,
            optimizer_fn=torch.optim.Adam,
            optimizer_args={'lr': 0.1,
                            'weight_decay': 1e-7})


Step:  10/400 Loss: 4.76438 Edge-Overlap: 0.385 Total-Time: 0
Step:  20/400 Loss: 4.07841 Edge-Overlap: 0.595 Total-Time: 0
Step:  30/400 Loss: 3.81889 Edge-Overlap: 0.641 Total-Time: 0
Step:  40/400 Loss: 3.68922 Edge-Overlap: 0.697 Total-Time: 0
Step:  50/400 Loss: 3.61537 Edge-Overlap: 0.730 Total-Time: 1
Step:  60/400 Loss: 3.56892 Edge-Overlap: 0.755 Total-Time: 1
Step:  70/400 Loss: 3.53666 Edge-Overlap: 0.777 Total-Time: 1
Step:  80/400 Loss: 3.51322 Edge-Overlap: 0.787 Total-Time: 1
Step:  90/400 Loss: 3.49563 Edge-Overlap: 0.802 Total-Time: 2


In [6]:
from utils import *

generated_graph = cell_model.sample_graph()
# k = 1.5
# generated_graph = cell_model.sample_graph_prime(k)
graph_prime = generated_graph.A
graph_prime = graph_post_process(graph_prime)
# print('here is graph_prime')
# print(graph_prime[graph_prime < 0])
# print(len(graph_prime))
# print(graph_prime)
# print(graph_prime[[0, 1, 2], [0, 0, 0]])

edge_index_prime = torch.tensor(graph_prime.nonzero(), dtype=torch.long)
x = embeddings
data_prime = Data(x=x, edge_index = edge_index_prime)
out = model(data_prime)
src, dst = edge_index_prime
score = (out[src] * out[dst]).sum(dim=-1)
weight = score.detach().numpy()
weight[weight <= 1] = 1
weight = np.rint(weight).astype(int)

def get_clique_candidates(lig_adjacency_matrix, k, l):
    graph = nx.from_numpy_matrix(lig_adjacency_matrix)
    cliques = nx.enumerate_all_cliques(graph)
    clique_candidates = []
    for clique in cliques:
        if len(clique) <= l:
            if len(clique) > k:
                clique_candidates.append(clique)
        else:
            break
    return clique_candidates


weighted_graph_prime = np.copy(graph_prime)
weighted_graph_prime[weighted_graph_prime.nonzero()] = weight
# lig_adjacency_matrix, lig_weighted_adjacency_matrix = sat_to_lig_adjacency_matrix(sat_instance, num_vars)

min_len = 1
clique_candidates = get_clique_candidates(graph_prime, min_len, max_len)
current_cliques = lazy_clique_edge_cover(np.copy(weighted_graph_prime), clique_candidates, int(num_clauses/1.5))
current_sat = cliques_to_sat(current_cliques)


# g = Glucose3(bootstrap_with=current_sat)
# g = Glucose3(bootstrap_with=sat_instance)
# %time print(g.solve())

14272
min: 1, max: 6
the clique numbers: 43334


In [7]:
features = [
        "clu. VIG",
        "clu. LIG",
        "mod. VIG",
        "mod. LIG",
        "mod. VCG",
        "mod. LCG"
]

metrics = eval_solution(current_sat, num_vars)
for feature, value in zip(features, metrics):
    print(f'{feature}: {value}')

clu. VIG: 0.4280694367530436
clu. LIG: 0.36201876409034894
mod. VIG: 0.4777515956333705
mod. LIG: 0.5460365994746994
mod. VCG: 0.7135670025622237
mod. LCG: 0.5524001227330368


In [8]:
metrics = eval_solution(sat_instance, num_vars)
for feature, value in zip(features, metrics):
    print(f'{feature}: {value}')

clu. VIG: 0.42752696354194286
clu. LIG: 0.3572546639124413
mod. VIG: 0.46900072424557754
mod. LIG: 0.5287190170088076
mod. VCG: 0.7828018018689825
mod. LCG: 0.7211265321275283


In [9]:
# init_lig = lig_adjacency_matrix
# generate_lig = graph_prime
# init_wlig = lig_weighted_adjacency_matrix
# generate_wlig = weighted_graph_prime
# formulas_lig, formulas_wlig = sat_to_lig_adjacency_matrix(current_sat, num_vars)



In [10]:
# import csv

# graphs = [init_lig, generate_lig, init_wlig, generate_wlig, formulas_wlig]
# graph_names = ['init_lig', 'generate_lig', 'init_wlig', 'generate_wlig', 'formulas_wlig']

# fileds = ['Source','Target','Type','Kind','Id','Label','Weight']
# with open('ssa2670-141-all-1.csv', 'w') as csvfile:
#     csvwriter = csv.writer(csvfile, delimiter=',')
#     csvwriter.writerow(fileds)
#     idx = 1
#     for graph_name, graph in zip(graph_names, graphs):
#         triu_adjacency_matrix = np.triu(graph)
#         x, y = triu_adjacency_matrix.nonzero()
#         for i, j in zip(x, y):
#             csvwriter.writerow([i, j, 'Undirected', graph_name, idx, graph_name, triu_adjacency_matrix[i][j]])
#             idx += 1
    

In [11]:
# generate_num = 50

# start_time = time.time()
# for idx in range(generate_num):
#     # print(idx)
#     generated_graph = cell_model.sample_graph()
#     graph_prime = generated_graph.A
#     graph_prime = graph_post_process(graph_prime)
#     # print(len(graph_prime))
#     # print(graph_prime)
#     # print(graph_prime[[0, 1, 2], [0, 0, 0]])

#     edge_index_prime = torch.tensor(graph_prime.nonzero(), dtype=torch.long)
#     data_prime = Data(x=x, edge_index = edge_index_prime)
#     out = model(data_prime)
#     src, dst = edge_index_prime
#     score = (out[src] * out[dst]).sum(dim=-1)
#     weight = score.detach().numpy()
#     weight[weight <= 1] = 1
#     weight = np.rint(weight).astype(int)
#     # print(f'histogram of inference weight: {np.histogram(weight, bins=[1, 2, 3, 4, 5, 6, 7])}')
#     # plt.hist(weight, bins=[1, 2, 3, 4, 5, 6, 7])

#     weighted_graph_prime = graph_prime
#     weighted_graph_prime[weighted_graph_prime.nonzero()] = weight
#     # lig_adjacency_matrix, lig_weighted_adjacency_matrix = sat_to_lig_adjacency_matrix(sat_instance, num_vars)
#     max_len = 8
#     clique_candidates = get_clique_candidates(graph_prime, max_len)
#     num_clauses = 377
#     num_vars = 91
#     # print(weighted_graph_prime.shape)
#     current_clique_idxs = lazy_clique_edge_cover(weighted_graph_prime, clique_candidates, num_clauses, num_vars)
#     current_cliques = [clique_candidates[idx] for idx in current_clique_idxs]
#     current_sat = cliques_to_sat(current_cliques)

#     path = f"./eval_formulas/ssa2670-141/generating-sat-{idx}.cnf"
#     with open(path, 'w') as f:
#         f.write('p cnf 91 377\n')
#         for clause in current_sat:
#             f.write(f"{' '.join([str(v) for v in clause])} 0\n")

# print(f'generating {generate_num} instances with time {time.time() - start_time}')

# Todo

- Optimize the optimal weighted coverage (OWC) algorithm
- Generate more graph for the ssa2670, compute the average metric
- Test the method on more instances
- *Test the solver performance*